In [1]:
import tensorflow as tf
import tensorflow.contrib.distributions as tfd

import edward as ed
from edward.models import Normal
from edward.models import MultivariateNormalFullCovariance
from edward.models import MultivariateNormalTriL

%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from IPython.display import display

plt.style.use('ggplot')

<h2> TTE with weibull </h2>
https://github.com/ragulpr/wtte-rnn/blob/master/python/wtte/wtte.py

In [6]:
def generate_weibull(A, B, C, shape, discrete_time):    
    W = np.sort(A * np.power(-np.log(np.random.uniform(0, 1, shape)), 1 / B))

    if discrete_time:
        C = np.floor(C)
        W = np.floor(W)

    U = np.less_equal(W, C) * 1.
    Y = np.minimum(W, C)
    return W, Y, U

In [5]:
def loglik_continuous(a, b, y_, u_, output_collection=()):

    ya = tf.div(y_ + 1e-35, a)  # Small optimization y/a

    loglik = tf.multiply(u_, tf.log(b) + tf.multiply(b, tf.log(ya))) - tf.pow(ya, b)
    tf.add_to_collection(output_collection, loglik)

    return(loglik)

<h2> univariate weibull </h2>

In [45]:
# simple univariate weibull distribution fit

y, y_censored, censor = generate_weibull(A=3, B=2, C=4, shape=(10, 1), discrete_time=False)

tf.reset_default_graph()

y_ = tf.placeholder(tf.float32, shape=(None, 1))
u_ = tf.placeholder(tf.float32, shape=(None, 1))

a=tf.Variable(tf.ones([1]))
b=tf.Variable(tf.ones([1]))


loglik = loglik_continuous(a, b, y_, u_)
loss = -tf.reduce_mean(loglik)

eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    loss_val, _, shape, scale = sess.run([loss, optimizer, a, b], feed_dict={y_:y_censored, u_:censor})
    if(i % 100 == 0):
        display('loss=' + str(loss_val) + ' shape=' + str(shape) + ' scale=' + str(scale))

'loss=1.868247 shape=[1.] scale=[1.]'

'loss=1.2848009 shape=[2.1616657] scale=[1.2802417]'

'loss=1.251549 shape=[2.4560914] scale=[1.385494]'

'loss=1.2456656 shape=[2.5815816] scale=[1.4115845]'

'loss=1.2443429 shape=[2.641484] scale=[1.4193795]'

'loss=1.2440084 shape=[2.6717012] scale=[1.4221767]'

'loss=1.2439185 shape=[2.6873784] scale=[1.4233367]'

'loss=1.2438937 shape=[2.6956317] scale=[1.4238693]'

'loss=1.2438867 shape=[2.700011] scale=[1.4241301]'

'loss=1.2438847 shape=[2.702344] scale=[1.424263]'

<h2> mixture univariate weibull </h2>

In [141]:

tf.reset_default_graph()
y1, y_censored1, censor1 = generate_weibull(A=3, B=2, C=4, shape=(10, 1), discrete_time=False)
y2, y_censored2, censor2 = generate_weibull(A=10, B=5, C=4, shape=(10, 1), discrete_time=False)
y_censored = np.expand_dims(np.append(y_censored1, y_censored2), 1)
censor = np.expand_dims(np.append(censor1, censor2), 1)

y_ = tf.placeholder(tf.float32, shape=(None, 1))
u_ = tf.placeholder(tf.float32, shape=(None, 1))

a=tf.Variable(tf.random_uniform(shape=(2,), maxval=10))
b=tf.Variable(tf.random_uniform(shape=(2,), maxval=10))
pi = tf.Variable(tf.truncated_normal([20, 2])) # data point probability to group
pi = tf.nn.softmax(pi)

loglik = loglik_continuous(a, b, y_, u_)
loss = -tf.reduce_mean(tf.multiply(pi, loglik), 0) # mixture probability

eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


for i in range(5000):
    loss_val, _, shape, scale, prob = sess.run([loss, optimizer, a, b, pi], feed_dict={y_:y_censored, u_:censor})
    if(i % 500 == 0):
        display('loss=' + str(loss_val) + ' shape=' + str(shape) + ' scale=' + str(scale))
        
display(prob)        

'loss=[0.46189833 2.198883  ] shape=[7.874148  7.7893615] scale=[2.3309457 7.7516365]'

'loss=[0.6283792  0.13120344] shape=[5.137437 6.096473] scale=[1.3242302 5.837031 ]'

'loss=[0.41077024 0.07065453] shape=[2.7941558 6.6805396] scale=[1.9916395 5.9262414]'

'loss=[0.36378193 0.04609757] shape=[2.5639148 7.1026163] scale=[2.125896 6.066879]'

'loss=[0.35193306 0.03378621] shape=[2.5290146 7.3975244] scale=[2.1638484 6.1892834]'

...

'loss=[0.34407702 0.02174372] shape=[2.507509  7.7989945] scale=[2.1900587 6.3841014]'

'loss=[0.34234244 0.0183852 ] shape=[2.5028582 7.946899 ] scale=[2.1959167 6.463494 ]'

'loss=[0.34115902 0.01589872] shape=[2.499695 8.072882] scale=[2.1999183 6.534179 ]'

'loss=[0.34030145 0.01398702] shape=[2.4974024 8.182362 ] scale=[2.2028165 6.5978227]'

array([[0.9982151 , 0.00178495],
       [0.99702185, 0.00297809],
       [0.00329662, 0.9967033 ],
       [0.99765164, 0.00234842],
       [0.99761295, 0.00238705],
       [0.99652356, 0.00347639],
       [0.00344064, 0.9965593 ],
       [0.99725753, 0.00274252],
       [0.998018  , 0.001982  ],
       [0.9972994 , 0.00270064],
       [0.00327043, 0.9967296 ],
       [0.00288563, 0.9971144 ],
       [0.00300068, 0.9969994 ],
       [0.00399829, 0.99600166],
       [0.00297207, 0.99702793],
       [0.00343036, 0.99656963],
       [0.00294305, 0.99705696],
       [0.00279209, 0.99720794],
       [0.0034337 , 0.9965663 ],
       [0.00324534, 0.9967546 ]], dtype=float32)